In [1]:
D.<x> =PolynomialRing(QQ)
DD=PolynomialRing(QQ,'q')
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]

def Delta(n):
    M = ModularForms(SL2Z, 12) 
    return DD(M.q_expansion_basis(n)[0])

def tau(n):
    return Delta(n+1)[n]
 
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})


print("okay")

okay


In [70]:
ls=[]
for n in [1..5]:ls=ls+[tau(n)]
print(ls)

[1, -24, 252, -1472, 4830]


In [71]:
def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)

print(RamanujanCongruenceSeries(5))



-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x


In [75]:
def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)


def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]

for k in [1..3]:
    print(RamanujanCongruenceSeries(n));print()
    print(recip(n)^k);print()
    rpc=recipPowerConstantTerm(k)
    test=order(2,rpc)-3*digitsum(2,k)
    print((rpc,test))
    
    print("----------------------------------------------------------")

-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-1 + 24 + 324*x + 3200*x^2 + 25650*x^3 + 176256*x^4 + 1056976*x^5 + O(x^6)

(24, 0)
----------------------------------------------------------
-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-2 + 48*x^-1 + 1224 + 21952*x + 309876*x^2 + 3657312*x^3 + 37435440*x^4 + O(x^5)

(1224, 0)
----------------------------------------------------------
-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-3 + 72*x^-2 + 2700*x^-1 + 70080 + 1412550*x + 23531040*x^2 + 336770016*x^3 + O(x^4)

(70080, 0)
----------------------------------------------------------


In [77]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..10]: # 100 < precision is necessary
        start=time.time()
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        finish=time.time()
        print((b,a,test,finish-start))
        if test!=0:ls=ls+[a]
    print()
    print("bad a's for trial ",b);print();print((ls))
    no=no+[(b,len(ls))]
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0.029613018035888672)
(1, 2, 0, 0.06286883354187012)
(1, 3, 0, 0.06935405731201172)
(1, 4, 0, 0.04050111770629883)
(1, 5, 0, 0.07944560050964355)
(1, 6, 0, 0.1292731761932373)
(1, 7, 0, 0.10259318351745605)
(1, 8, 0, 0.09853911399841309)
(1, 9, 0, 0.12269401550292969)
(1, 10, 0, 0.11371517181396484)

bad a's for trial  1

[]
------------------------------------------------------------------
no:

[(1, 0)]


In [80]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..100]: # 100 < precision is necessary
        print((b,a))
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        tests=tests+[test]
    print("trial",b)
    print(tests)
    print("-----------------------------------------------------------------------")
        
        


(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(1, 20)
(1, 21)
(1, 22)
(1, 23)
(1, 24)
(1, 25)
(1, 26)
(1, 27)
(1, 28)
(1, 29)
(1, 30)
(1, 31)
(1, 32)
(1, 33)
(1, 34)
(1, 35)
(1, 36)
(1, 37)
(1, 38)
(1, 39)
(1, 40)
(1, 41)
(1, 42)
(1, 43)
(1, 44)
(1, 45)
(1, 46)
(1, 47)
(1, 48)
(1, 49)
(1, 50)
(1, 51)
(1, 52)
(1, 53)
(1, 54)
(1, 55)
(1, 56)
(1, 57)
(1, 58)
(1, 59)
(1, 60)
(1, 61)
(1, 62)
(1, 63)
(1, 64)
(1, 65)
(1, 66)
(1, 67)
(1, 68)
(1, 69)
(1, 70)
(1, 71)
(1, 72)
(1, 73)
(1, 74)
(1, 75)
(1, 76)
(1, 77)
(1, 78)
(1, 79)
(1, 80)
(1, 81)
(1, 82)
(1, 83)
(1, 84)
(1, 85)
(1, 86)
(1, 87)
(1, 88)
(1, 89)
(1, 90)
(1, 91)
(1, 92)
(1, 93)
(1, 94)
(1, 95)
(1, 96)
(1, 97)
(1, 98)
(1, 99)
(1, 100)
trial 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [2]:
import random
import time


def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
for b in [1..10]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        print((b,a))
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        tests=tests+[test]
    print();print("trial",b)
    print();print(tests)
    print("-----------------------------------------------------------------------")


(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(1, 20)
(1, 21)
(1, 22)
(1, 23)
(1, 24)
(1, 25)
(1, 26)
(1, 27)
(1, 28)
(1, 29)
(1, 30)
(1, 31)
(1, 32)
(1, 33)
(1, 34)
(1, 35)
(1, 36)
(1, 37)
(1, 38)
(1, 39)
(1, 40)
(1, 41)
(1, 42)
(1, 43)
(1, 44)
(1, 45)
(1, 46)
(1, 47)
(1, 48)
(1, 49)
(1, 50)
(1, 51)
(1, 52)
(1, 53)
(1, 54)
(1, 55)
(1, 56)
(1, 57)
(1, 58)
(1, 59)
(1, 60)
(1, 61)
(1, 62)
(1, 63)
(1, 64)
(1, 65)
(1, 66)
(1, 67)
(1, 68)
(1, 69)
(1, 70)
(1, 71)
(1, 72)
(1, 73)
(1, 74)
(1, 75)
(1, 76)
(1, 77)
(1, 78)
(1, 79)
(1, 80)
(1, 81)
(1, 82)
(1, 83)
(1, 84)
(1, 85)
(1, 86)
(1, 87)
(1, 88)
(1, 89)
(1, 90)
(1, 91)
(1, 92)
(1, 93)
(1, 94)
(1, 95)
(1, 96)
(1, 97)
(1, 98)
(1, 99)
(1, 100)
(1, 101)
(1, 102)
(1, 103)
(1, 104)
(1, 105)
(1, 106)
(1, 107)
(1, 108)
(1, 109)
(1, 110)
(1, 111)
(1, 112)
(1, 113)
(1, 114)
(1, 115)
(1, 116)
(1, 117)
(1, 118)
(1, 119)
(1, 120)
(1, 121)
(1, 122)
(1, 123)


(2, 260)
(2, 261)
(2, 262)
(2, 263)
(2, 264)
(2, 265)
(2, 266)
(2, 267)
(2, 268)
(2, 269)
(2, 270)
(2, 271)
(2, 272)
(2, 273)
(2, 274)
(2, 275)
(2, 276)
(2, 277)
(2, 278)
(2, 279)
(2, 280)
(2, 281)
(2, 282)
(2, 283)
(2, 284)
(2, 285)
(2, 286)
(2, 287)
(2, 288)
(2, 289)
(2, 290)
(2, 291)
(2, 292)
(2, 293)
(2, 294)
(2, 295)
(2, 296)
(2, 297)
(2, 298)
(2, 299)
(2, 300)
(2, 301)
(2, 302)
(2, 303)
(2, 304)
(2, 305)
(2, 306)
(2, 307)
(2, 308)
(2, 309)
(2, 310)
(2, 311)
(2, 312)
(2, 313)
(2, 314)
(2, 315)
(2, 316)
(2, 317)
(2, 318)
(2, 319)
(2, 320)
(2, 321)
(2, 322)
(2, 323)
(2, 324)
(2, 325)
(2, 326)
(2, 327)
(2, 328)
(2, 329)
(2, 330)
(2, 331)
(2, 332)
(2, 333)
(2, 334)
(2, 335)
(2, 336)
(2, 337)
(2, 338)
(2, 339)
(2, 340)
(2, 341)
(2, 342)
(2, 343)
(2, 344)
(2, 345)
(2, 346)
(2, 347)
(2, 348)
(2, 349)
(2, 350)
(2, 351)
(2, 352)
(2, 353)
(2, 354)
(2, 355)
(2, 356)
(2, 357)
(2, 358)
(2, 359)
(2, 360)
(2, 361)
(2, 362)
(2, 363)
(2, 364)
(2, 365)
(2, 366)
(2, 367)
(2, 368)
(2, 369)
(2, 370)
(

(3, 340)
(3, 341)
(3, 342)
(3, 343)
(3, 344)
(3, 345)
(3, 346)
(3, 347)
(3, 348)
(3, 349)
(3, 350)
(3, 351)
(3, 352)
(3, 353)
(3, 354)
(3, 355)
(3, 356)
(3, 357)
(3, 358)
(3, 359)
(3, 360)
(3, 361)
(3, 362)
(3, 363)
(3, 364)
(3, 365)
(3, 366)
(3, 367)
(3, 368)
(3, 369)
(3, 370)
(3, 371)
(3, 372)
(3, 373)
(3, 374)
(3, 375)
(3, 376)
(3, 377)
(3, 378)
(3, 379)
(3, 380)
(3, 381)
(3, 382)
(3, 383)
(3, 384)
(3, 385)
(3, 386)
(3, 387)
(3, 388)
(3, 389)
(3, 390)
(3, 391)
(3, 392)
(3, 393)
(3, 394)
(3, 395)
(3, 396)
(3, 397)
(3, 398)
(3, 399)
(3, 400)
(3, 401)
(3, 402)
(3, 403)
(3, 404)
(3, 405)
(3, 406)
(3, 407)
(3, 408)
(3, 409)
(3, 410)
(3, 411)
(3, 412)
(3, 413)
(3, 414)
(3, 415)
(3, 416)
(3, 417)
(3, 418)
(3, 419)
(3, 420)
(3, 421)
(3, 422)
(3, 423)
(3, 424)
(3, 425)
(3, 426)
(3, 427)
(3, 428)
(3, 429)
(3, 430)
(3, 431)
(3, 432)
(3, 433)
(3, 434)
(3, 435)
(3, 436)
(3, 437)
(3, 438)
(3, 439)
(3, 440)
(3, 441)
(3, 442)
(3, 443)
(3, 444)
(3, 445)
(3, 446)
(3, 447)
(3, 448)
(3, 449)
(3, 450)
(

(4, 253)
(4, 254)
(4, 255)
(4, 256)
(4, 257)
(4, 258)
(4, 259)
(4, 260)
(4, 261)
(4, 262)
(4, 263)
(4, 264)
(4, 265)
(4, 266)
(4, 267)
(4, 268)
(4, 269)
(4, 270)
(4, 271)
(4, 272)
(4, 273)
(4, 274)
(4, 275)
(4, 276)
(4, 277)
(4, 278)
(4, 279)
(4, 280)
(4, 281)
(4, 282)
(4, 283)
(4, 284)
(4, 285)
(4, 286)
(4, 287)
(4, 288)
(4, 289)
(4, 290)
(4, 291)
(4, 292)
(4, 293)
(4, 294)
(4, 295)
(4, 296)
(4, 297)
(4, 298)
(4, 299)
(4, 300)
(4, 301)
(4, 302)
(4, 303)
(4, 304)
(4, 305)
(4, 306)
(4, 307)
(4, 308)
(4, 309)
(4, 310)
(4, 311)
(4, 312)
(4, 313)
(4, 314)
(4, 315)
(4, 316)
(4, 317)
(4, 318)
(4, 319)
(4, 320)
(4, 321)
(4, 322)
(4, 323)
(4, 324)
(4, 325)
(4, 326)
(4, 327)
(4, 328)
(4, 329)
(4, 330)
(4, 331)
(4, 332)
(4, 333)
(4, 334)
(4, 335)
(4, 336)
(4, 337)
(4, 338)
(4, 339)
(4, 340)
(4, 341)
(4, 342)
(4, 343)
(4, 344)
(4, 345)
(4, 346)
(4, 347)
(4, 348)
(4, 349)
(4, 350)
(4, 351)
(4, 352)
(4, 353)
(4, 354)
(4, 355)
(4, 356)
(4, 357)
(4, 358)
(4, 359)
(4, 360)
(4, 361)
(4, 362)
(4, 363)
(

(5, 5)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(5, 16)
(5, 17)
(5, 18)
(5, 19)
(5, 20)
(5, 21)
(5, 22)
(5, 23)
(5, 24)
(5, 25)
(5, 26)
(5, 27)
(5, 28)
(5, 29)
(5, 30)
(5, 31)
(5, 32)
(5, 33)
(5, 34)
(5, 35)
(5, 36)
(5, 37)
(5, 38)
(5, 39)
(5, 40)
(5, 41)
(5, 42)
(5, 43)
(5, 44)
(5, 45)
(5, 46)
(5, 47)
(5, 48)
(5, 49)
(5, 50)
(5, 51)
(5, 52)
(5, 53)
(5, 54)
(5, 55)
(5, 56)
(5, 57)
(5, 58)
(5, 59)
(5, 60)
(5, 61)
(5, 62)
(5, 63)
(5, 64)
(5, 65)
(5, 66)
(5, 67)
(5, 68)
(5, 69)
(5, 70)
(5, 71)
(5, 72)
(5, 73)
(5, 74)
(5, 75)
(5, 76)
(5, 77)
(5, 78)
(5, 79)
(5, 80)
(5, 81)
(5, 82)
(5, 83)
(5, 84)
(5, 85)
(5, 86)
(5, 87)
(5, 88)
(5, 89)
(5, 90)
(5, 91)
(5, 92)
(5, 93)
(5, 94)
(5, 95)
(5, 96)
(5, 97)
(5, 98)
(5, 99)
(5, 100)
(5, 101)
(5, 102)
(5, 103)
(5, 104)
(5, 105)
(5, 106)
(5, 107)
(5, 108)
(5, 109)
(5, 110)
(5, 111)
(5, 112)
(5, 113)
(5, 114)
(5, 115)
(5, 116)
(5, 117)
(5, 118)
(5, 119)
(5, 120)
(5, 121)
(5, 122)
(5, 123)
(5, 124)
(5, 125)
(5, 126)
(5

(6, 5)
(6, 6)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(6, 13)
(6, 14)
(6, 15)
(6, 16)
(6, 17)
(6, 18)
(6, 19)
(6, 20)
(6, 21)
(6, 22)
(6, 23)
(6, 24)
(6, 25)
(6, 26)
(6, 27)
(6, 28)
(6, 29)
(6, 30)
(6, 31)
(6, 32)
(6, 33)
(6, 34)
(6, 35)
(6, 36)
(6, 37)
(6, 38)
(6, 39)
(6, 40)
(6, 41)
(6, 42)
(6, 43)
(6, 44)
(6, 45)
(6, 46)
(6, 47)
(6, 48)
(6, 49)
(6, 50)
(6, 51)
(6, 52)
(6, 53)
(6, 54)
(6, 55)
(6, 56)
(6, 57)
(6, 58)
(6, 59)
(6, 60)
(6, 61)
(6, 62)
(6, 63)
(6, 64)
(6, 65)
(6, 66)
(6, 67)
(6, 68)
(6, 69)
(6, 70)
(6, 71)
(6, 72)
(6, 73)
(6, 74)
(6, 75)
(6, 76)
(6, 77)
(6, 78)
(6, 79)
(6, 80)
(6, 81)
(6, 82)
(6, 83)
(6, 84)
(6, 85)
(6, 86)
(6, 87)
(6, 88)
(6, 89)
(6, 90)
(6, 91)
(6, 92)
(6, 93)
(6, 94)
(6, 95)
(6, 96)
(6, 97)
(6, 98)
(6, 99)
(6, 100)
(6, 101)
(6, 102)
(6, 103)
(6, 104)
(6, 105)
(6, 106)
(6, 107)
(6, 108)
(6, 109)
(6, 110)
(6, 111)
(6, 112)
(6, 113)
(6, 114)
(6, 115)
(6, 116)
(6, 117)
(6, 118)
(6, 119)
(6, 120)
(6, 121)
(6, 122)
(6, 123)
(6, 124)
(6, 125)
(6, 126)
(6

(7, 5)
(7, 6)
(7, 7)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(7, 16)
(7, 17)
(7, 18)
(7, 19)
(7, 20)
(7, 21)
(7, 22)
(7, 23)
(7, 24)
(7, 25)
(7, 26)
(7, 27)
(7, 28)
(7, 29)
(7, 30)
(7, 31)
(7, 32)
(7, 33)
(7, 34)
(7, 35)
(7, 36)
(7, 37)
(7, 38)
(7, 39)
(7, 40)
(7, 41)
(7, 42)
(7, 43)
(7, 44)
(7, 45)
(7, 46)
(7, 47)
(7, 48)
(7, 49)
(7, 50)
(7, 51)
(7, 52)
(7, 53)
(7, 54)
(7, 55)
(7, 56)
(7, 57)
(7, 58)
(7, 59)
(7, 60)
(7, 61)
(7, 62)
(7, 63)
(7, 64)
(7, 65)
(7, 66)
(7, 67)
(7, 68)
(7, 69)
(7, 70)
(7, 71)
(7, 72)
(7, 73)
(7, 74)
(7, 75)
(7, 76)
(7, 77)
(7, 78)
(7, 79)
(7, 80)
(7, 81)
(7, 82)
(7, 83)
(7, 84)
(7, 85)
(7, 86)
(7, 87)
(7, 88)
(7, 89)
(7, 90)
(7, 91)
(7, 92)
(7, 93)
(7, 94)
(7, 95)
(7, 96)
(7, 97)
(7, 98)
(7, 99)
(7, 100)
(7, 101)
(7, 102)
(7, 103)
(7, 104)
(7, 105)
(7, 106)
(7, 107)
(7, 108)
(7, 109)
(7, 110)
(7, 111)
(7, 112)
(7, 113)
(7, 114)
(7, 115)
(7, 116)
(7, 117)
(7, 118)
(7, 119)
(7, 120)
(7, 121)
(7, 122)
(7, 123)
(7, 124)
(7, 125)
(7, 126)
(7

(8, 5)
(8, 6)
(8, 7)
(8, 8)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
(8, 15)
(8, 16)
(8, 17)
(8, 18)
(8, 19)
(8, 20)
(8, 21)
(8, 22)
(8, 23)
(8, 24)
(8, 25)
(8, 26)
(8, 27)
(8, 28)
(8, 29)
(8, 30)
(8, 31)
(8, 32)
(8, 33)
(8, 34)
(8, 35)
(8, 36)
(8, 37)
(8, 38)
(8, 39)
(8, 40)
(8, 41)
(8, 42)
(8, 43)
(8, 44)
(8, 45)
(8, 46)
(8, 47)
(8, 48)
(8, 49)
(8, 50)
(8, 51)
(8, 52)
(8, 53)
(8, 54)
(8, 55)
(8, 56)
(8, 57)
(8, 58)
(8, 59)
(8, 60)
(8, 61)
(8, 62)
(8, 63)
(8, 64)
(8, 65)
(8, 66)
(8, 67)
(8, 68)
(8, 69)
(8, 70)
(8, 71)
(8, 72)
(8, 73)
(8, 74)
(8, 75)
(8, 76)
(8, 77)
(8, 78)
(8, 79)
(8, 80)
(8, 81)
(8, 82)
(8, 83)
(8, 84)
(8, 85)
(8, 86)
(8, 87)
(8, 88)
(8, 89)
(8, 90)
(8, 91)
(8, 92)
(8, 93)
(8, 94)
(8, 95)
(8, 96)
(8, 97)
(8, 98)
(8, 99)
(8, 100)
(8, 101)
(8, 102)
(8, 103)
(8, 104)
(8, 105)
(8, 106)
(8, 107)
(8, 108)
(8, 109)
(8, 110)
(8, 111)
(8, 112)
(8, 113)
(8, 114)
(8, 115)
(8, 116)
(8, 117)
(8, 118)
(8, 119)
(8, 120)
(8, 121)
(8, 122)
(8, 123)
(8, 124)
(8, 125)
(8, 126)
(8

(9, 5)
(9, 6)
(9, 7)
(9, 8)
(9, 9)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(9, 16)
(9, 17)
(9, 18)
(9, 19)
(9, 20)
(9, 21)
(9, 22)
(9, 23)
(9, 24)
(9, 25)
(9, 26)
(9, 27)
(9, 28)
(9, 29)
(9, 30)
(9, 31)
(9, 32)
(9, 33)
(9, 34)
(9, 35)
(9, 36)
(9, 37)
(9, 38)
(9, 39)
(9, 40)
(9, 41)
(9, 42)
(9, 43)
(9, 44)
(9, 45)
(9, 46)
(9, 47)
(9, 48)
(9, 49)
(9, 50)
(9, 51)
(9, 52)
(9, 53)
(9, 54)
(9, 55)
(9, 56)
(9, 57)
(9, 58)
(9, 59)
(9, 60)
(9, 61)
(9, 62)
(9, 63)
(9, 64)
(9, 65)
(9, 66)
(9, 67)
(9, 68)
(9, 69)
(9, 70)
(9, 71)
(9, 72)
(9, 73)
(9, 74)
(9, 75)
(9, 76)
(9, 77)
(9, 78)
(9, 79)
(9, 80)
(9, 81)
(9, 82)
(9, 83)
(9, 84)
(9, 85)
(9, 86)
(9, 87)
(9, 88)
(9, 89)
(9, 90)
(9, 91)
(9, 92)
(9, 93)
(9, 94)
(9, 95)
(9, 96)
(9, 97)
(9, 98)
(9, 99)
(9, 100)
(9, 101)
(9, 102)
(9, 103)
(9, 104)
(9, 105)
(9, 106)
(9, 107)
(9, 108)
(9, 109)
(9, 110)
(9, 111)
(9, 112)
(9, 113)
(9, 114)
(9, 115)
(9, 116)
(9, 117)
(9, 118)
(9, 119)
(9, 120)
(9, 121)
(9, 122)
(9, 123)
(9, 124)
(9, 125)
(9, 126)
(9

(10, 6)
(10, 7)
(10, 8)
(10, 9)
(10, 10)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(10, 16)
(10, 17)
(10, 18)
(10, 19)
(10, 20)
(10, 21)
(10, 22)
(10, 23)
(10, 24)
(10, 25)
(10, 26)
(10, 27)
(10, 28)
(10, 29)
(10, 30)
(10, 31)
(10, 32)
(10, 33)
(10, 34)
(10, 35)
(10, 36)
(10, 37)
(10, 38)
(10, 39)
(10, 40)
(10, 41)
(10, 42)
(10, 43)
(10, 44)
(10, 45)
(10, 46)
(10, 47)
(10, 48)
(10, 49)
(10, 50)
(10, 51)
(10, 52)
(10, 53)
(10, 54)
(10, 55)
(10, 56)
(10, 57)
(10, 58)
(10, 59)
(10, 60)
(10, 61)
(10, 62)
(10, 63)
(10, 64)
(10, 65)
(10, 66)
(10, 67)
(10, 68)
(10, 69)
(10, 70)
(10, 71)
(10, 72)
(10, 73)
(10, 74)
(10, 75)
(10, 76)
(10, 77)
(10, 78)
(10, 79)
(10, 80)
(10, 81)
(10, 82)
(10, 83)
(10, 84)
(10, 85)
(10, 86)
(10, 87)
(10, 88)
(10, 89)
(10, 90)
(10, 91)
(10, 92)
(10, 93)
(10, 94)
(10, 95)
(10, 96)
(10, 97)
(10, 98)
(10, 99)
(10, 100)
(10, 101)
(10, 102)
(10, 103)
(10, 104)
(10, 105)
(10, 106)
(10, 107)
(10, 108)
(10, 109)
(10, 110)
(10, 111)
(10, 112)
(10, 113)
(10, 114)
(10, 115